# Laboratory

**Author: @THEFFTKID**

So far, we have trained sensation-based models to tune the various parameters. In order to perform a better search over the hyperparameter space we use the Hyperopt library, seeking to minimize the negative of the acurracy.

In [1]:
cd ~/downloads/Pedro-Pineapple/forecasting

/home/aargaez/downloads/Pedro-Pineapple/forecasting


In [2]:
import mlflow
from mlflow import MlflowClient
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

In [3]:
# Flag experiment.
new_experiment = True
# Name
experiment_name = "A2C"
# Description.
experiment_description = (
    "Index - Stock forecasting project."
    "This experiment contains the models using AC2 policy."
)
# Tags
experiment_tags = {
    "project_name": "Continous update policy",
    "mlflow.note.content": experiment_description,
}

In [4]:
client = MlflowClient(tracking_uri="http://127.0.0.1:8080")

In [5]:
if new_experiment:
    # Creates the MLflow experiment.
    produce_experiment = client.create_experiment(
        name=experiment_name,
        tags=experiment_tags
    )

In [6]:
import pandas as pd
import environments
import numpy as np
import evaluation
import utils

from dynamic_threshold import define_threshold
from stable_baselines3 import A2C
from dataclasses import dataclass
from typing import Dict, Any, List
from json import dumps

2023-11-22 21:04:00.275832: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-22 21:04:02.491951: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-22 21:04:07.480848: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [7]:
# Use the fluent API to set the tracking uri and the active experiment.
mlflow.set_tracking_uri("http://127.0.0.1:8080")

# Sets the current active experiment to the "index_forecast" experiment and returns the experiment metadata.
experiment = mlflow.set_experiment(experiment_name)

# Define an artifact path that the model will be saved to.
artifact_path = "a2c_index"

In [8]:
def train_experiment(parameters: Dict[str, Any], features: List[str]):
    """
    Wrapper function for MLflow model versioning
    """
    # Read data.
    data = pd.read_csv(filepath_or_buffer=parameters['data_path'])
    # Cast columns.
    data.columns = data.columns.str.lower()

    data['labels'], data['perc_relative_diff'] = evaluation.create_labels(
        x=data['close'],
        labels=[0, 1, 2],
        perc_bounds=[
            parameters['lower_threshold'],
            parameters['upper_threshold']
        ]
    )
    
    # Split dataset into train and test.
    df_train, df_test = evaluation.data_splitter(
        raw_data=data,
        proportion=parameters['proportion']
    )

    # Create the environment.
    env = environments.Forecasting(
        df=df_train,
        series_features=features,
        window_size=parameters['window_size'],
        lower_threshold=parameters['lower_threshold'],
        upper_threshold=parameters['upper_threshold']
    )

    # Train environment.
    model = A2C('MlpPolicy', env, verbose=0)

    # Train model with MLflow tracking
    with mlflow.start_run(nested=True):
        # Learn.
        model.learn(total_timesteps=parameters['time_steps'])

        # Initialize a new environment with test-set.
        env_test = environments.Forecasting(
            df=df_test,
            series_features=features,
            window_size=parameters['window_size'],
            lower_threshold=parameters['lower_threshold'],
            upper_threshold=parameters['upper_threshold']
        )

        # Reset environment.
        observation, info = env_test.reset(
            upper_threshold=env.up_threshold,
            lower_threshold=env.low_threshold
        )

        while True: 
            observation = observation[np.newaxis, ...]
            action, states = model.predict(observation)
            observation, rewards, done, truncated, info = env_test.step(action)
            if done or truncated:
                print('info', info, '\n')
                break
        
        y_true = df_test['labels'].to_numpy()[parameters['window_size'] + 1:]
        y_pred = np.concatenate(env_test.actions_history)

        performance = evaluation.evaluation_metrics(
            y_true=y_true,
            y_pred=y_pred,
            target_names=['down', 'no', 'up']
        )

        accuracy = performance['accuracy']

        # print(dumps(performance, indent=4))

        # Logging params and metrics to MLflow.
        metrics = utils.flatten_dict(d=performance)

        # Log the parameters used for the model fit.
        # mlflow.log_params(params=dataclasses.asdict(parameters))
        mlflow.log_params(params=parameters)

        # Log the error metrics that were calculated during validation.
        mlflow.log_metrics(metrics=metrics)

        # TODO: Log an instance of the trained model for later use.
        
        return {"loss": -accuracy, "status": STATUS_OK}

In [9]:
def objective(params):
    # MLflow will track the parameters and results for each run.
    result = train_experiment(
        parameters=params,
        features=[
            'open',
            'high',
            'low',
            'volume_roc',
            'n10_rolling_mean',
            'n10_weighted_rolling_mean',
            'momentum',
            'close',
            'nday_tendency_removal'
        ]
    )
    return result

In [10]:
# Search space.
space = {
    "time_steps":  hp.randint("time_steps", 500, 1500),
    "window_size": hp.randint("window_size", 10, 15),
    "lower_threshold": hp.uniform("lower_threshold", 0.4, 0.5),
    "upper_threshold": hp.uniform("upper_threshold", 0.5, 0.6),
    "proportion": hp.uniform("proportion", 0.7, 0.9),
    "data_path": hp.choice("data_path", (
        '/home/aargaez/downloads/Pedro-Pineapple/data/SPY_20022006.csv',
        '/home/aargaez/downloads/Pedro-Pineapple/data/SPY_20122016.csv',
        )
    )
}

In [11]:
# Run for save the model.
with mlflow.start_run():
    # Conduct the hyperparameter search using Hyperopt.
    trials = Trials()

    best = fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=12,
        trials=trials,
    )

    # Fetch the details of the best run
    best_run = sorted(trials.results, key=lambda x: x["loss"])[0]

    # Log the best parameters, loss, and model
    # mlflow.log_params(best)
    # mlflow.log_metric("rmse", best_run["loss"])
    # mlflow.tensorflow.log_model(best["model"], "model", signature=signature)

    # Best parameters.
    print(f"Best parameters: {best}")
    print(f"Best rmse: {best_run['loss']}")

info                                                  
{'total_reward': 70.25005199999998}                   
info                                                                              
{'total_reward': 69.10005900000002}                                               
info                                                                              
{'total_reward': 85.2800149999999}                                               
info                                                                             
{'total_reward': 103.54997499999999}                                             
 33%|███▎      | 4/12 [00:21<00:40,  5.03s/trial, best loss: -0.4474885844748858]

/home/aargaez/downloads/Pedro-Pineapple/venvpp/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))

/home/aargaez/downloads/Pedro-Pineapple/venvpp/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))

/home/aargaez/downloads/Pedro-Pineapple/venvpp/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(

info                                                                             
{'total_reward': 138.50998099999984}                                             
info                                                                              
{'total_reward': 187.11005200000002}                                              
info                                                                              
{'total_reward': 60.00001200000003}                                               
info                                                                              
{'total_reward': 57.77001900000006}                                               
info                                                                              
{'total_reward': 93.93000799999984}                                               
info                                                                              
{'total_reward': 142.8599719999999}                                               
info  

/home/aargaez/downloads/Pedro-Pineapple/venvpp/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))

/home/aargaez/downloads/Pedro-Pineapple/venvpp/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))

/home/aargaez/downloads/Pedro-Pineapple/venvpp/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(